In [74]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [75]:
data = pd.read_csv('./data/data.csv')

Drop the columns we won't need.

In [76]:
data.drop(['PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'GP', 'W', 'L', 'SEASON', 'PK', 'PK_2', 'PK_3'], axis=1, inplace=True)

### Train-test split.

We will keep data from years 2022 and 2023 for later testing.

In [77]:
# data.dtypes

In [78]:
data_train = data[(data['YEAR_MVP'] != 2022) & (data['YEAR_MVP'] != 2023)]

In [79]:
data_test = data[(data['YEAR_MVP'] == 2022) | (data['YEAR_MVP'] == 2023)]

In [80]:
data_train.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_12032\440629547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop('YEAR_MVP', axis=1, inplace=True)


In [81]:
# data_train

In [82]:
# data_test

### X-y split.

In [83]:
X_train = data_train.drop('Share', axis=1)
y_train = data_train['Share']

##### Scaling.

In [84]:
from sklearn.preprocessing import MinMaxScaler
scaler= MinMaxScaler()
scaler.fit(X_train)

MinMaxScaler()

In [85]:
X_train_tr = pd.DataFrame(scaler.transform(X_train), index=X_train.index, columns=X_train.columns)

In [86]:
X_train_tr.isna().sum().sum()

0

In [87]:
y_train

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10817    0.0
10818    0.0
10819    0.0
10820    0.0
10821    0.0
Name: Share, Length: 10822, dtype: float64

### Model pipeline.

In [88]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

model1 = RandomForestRegressor()
model2 = GradientBoostingRegressor()
model3 = SVR()
model4 = XGBRegressor()

model_pipeline = [model1, model2, model3, model4]
model_names = ['Random Forest Regressor', 'Gradient Boosting Regressor', 'SVR', 'XGBoost Regressor']

scores = {}
for model, model_name in zip(model_pipeline, model_names):
    mean_score = np.mean(cross_val_score(model, X_train_tr, y_train, cv=5))
    scores[model_name] = mean_score
print(scores)

{'Random Forest Regressor': 0.5799619788112785, 'Gradient Boosting Regressor': 0.5036566108641406, 'SVR': 0.1415444137345662, 'XGBoost Regressor': 0.5190842790728889}


### Hyperparameter search (r2 score).

In [89]:
from sklearn.model_selection import GridSearchCV
rf_model = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, scoring='r2', n_jobs=-1)

grid_search.fit(X_train_tr, y_train)

best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

best_rf_model = grid_search.best_estimator_

r2_score_train = best_rf_model.score(X_train_tr, y_train)
print("R^2:", r2_score_train)

Best hyperparameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 200}
R^2: 0.9159893510954521


### Hyperparameter search (mean absolute error).

In [90]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_model_2 = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['auto', 'sqrt']
}

grid_search = GridSearchCV(estimator=rf_model_2, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)

grid_search.fit(X_train_tr, y_train)

best_params = grid_search.best_params_
print("Best hyperparameters:", best_params)

best_rf_model_2 = grid_search.best_estimator_

neg_mae_train = grid_search.best_score_
mae_train = -neg_mae_train
print("MAE train:", mae_train)

c:\Users\dcast\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best hyperparameters: {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}
MAE train: 0.004902347374223616


### Test year 2022.

With rf_model.

In [91]:
data_test_2022 = data_test[data_test['YEAR_MVP'] == 2022]
data_test_2022.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_12032\1205681605.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_2022.drop('YEAR_MVP', axis=1, inplace=True)


In [92]:
X_test_2022 = data_test_2022.drop('Share', axis=1)
y_test_2022 = data_test_2022['Share']

In [93]:
X_test_2022_tr = pd.DataFrame(scaler.transform(X_test_2022), index=X_test_2022.index, columns=X_test_2022.columns)

In [94]:
pred_2022 = best_rf_model.predict(X_test_2022_tr)

In [95]:
pred_2022 = pd.Series(pred_2022, index=X_test_2022.index, name='Share_PRED')

In [96]:
data_or = pd.read_csv('./data/data.csv')
data_2022 = data_or[data_or['YEAR_MVP'] == 2022]

In [97]:
output_2022 = pd.concat([data_2022, pred_2022], axis=1)

In [98]:
output_2022.isna().sum().sum()

0

In [99]:
output_2022.sort_values(by='Share_PRED', ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,Share_PRED
11271,203999,Nikola Jokic,Nikola,1610612743,DEN,27.0,74,46,28,0.622,33.5,10.3,17.7,0.583,1.3,3.9,0.337,5.1,6.3,0.810,2.8,11.0,13.8,7.9,3.8,1.5,0.9,0.9,2.6,6.5,27.1,6.0,58.6,66,19,54.7,2021-22,203999_2021-22,115.2,117.3,117.3,108.1,108.9,108.9,7.1,8.4,8.4,0.388,2.08,24.8,0.090,0.313,0.209,11.9,11.8,0.620,0.661,0.309,0.316,100.69,99.46,82.88,99.46,0.230,5136,10.3,17.7,1610612743_2021-22,0.585,6,2,2022,203999_2022,0.875,0.544599
11028,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,27.0,67,45,22,0.672,32.9,10.3,18.6,0.553,1.1,3.6,0.293,8.3,11.4,0.722,2.0,9.6,11.6,5.8,3.3,1.1,1.4,0.9,3.2,8.8,29.9,5.9,56.5,46,4,53.2,2021-22,203507_2021-22,115.5,115.9,115.9,107.8,107.9,107.9,7.7,8.0,8.0,0.304,1.77,17.8,0.060,0.267,0.168,10.0,10.0,0.582,0.633,0.340,0.347,103.36,103.12,85.93,103.12,0.210,4743,10.3,18.6,1610612749_2021-22,0.622,3,1,2022,203507_2022,0.595,0.535051
11117,203954,Joel Embiid,Joel,1610612755,PHI,28.0,68,45,23,0.662,33.8,9.8,19.6,0.499,1.4,3.7,0.371,9.6,11.8,0.814,2.1,9.6,11.7,4.2,3.1,1.1,1.5,1.2,2.7,8.5,30.6,5.4,55.5,46,2,53.0,2021-22,203954_2021-22,114.6,115.6,115.6,106.9,107.8,107.8,7.7,7.9,7.9,0.236,1.33,13.0,0.068,0.273,0.176,9.8,9.8,0.534,0.616,0.375,0.380,97.81,96.96,80.80,96.96,0.212,4640,9.8,19.6,1610612755_2021-22,0.622,4,2,2022,203954_2022,0.706,0.511910
11205,1629029,Luka Doncic,Luka,1610612742,DAL,23.0,65,44,21,0.677,35.4,9.9,21.6,0.457,3.1,8.8,0.353,5.6,7.5,0.744,0.9,8.3,9.1,8.7,4.5,1.2,0.6,0.7,2.2,6.2,28.4,2.2,53.1,44,10,52.8,2021-22,1629029_2021-22,112.8,113.8,113.8,108.6,110.3,110.3,4.1,3.5,3.5,0.458,1.95,22.9,0.025,0.230,0.129,11.8,11.8,0.529,0.571,0.368,0.375,97.37,96.18,80.15,96.18,0.191,4604,9.9,21.6,1610612742_2021-22,0.634,4,2,2022,1629029_2022,0.146,0.355266
11168,201142,Kevin Durant,Kevin,1610612751,BKN,33.0,55,36,19,0.655,37.2,10.5,20.3,0.518,2.1,5.5,0.383,6.8,7.4,0.910,0.5,6.9,7.4,6.4,3.5,0.9,0.9,0.7,2.1,5.9,29.9,4.9,50.3,18,4,49.4,2021-22,201142_2021-22,114.6,116.2,116.2,107.6,109.7,109.7,6.9,6.5,6.5,0.274,1.84,19.1,0.015,0.166,0.097,10.4,10.4,0.570,0.634,0.310,0.314,102.88,101.17,84.31,101.17,0.180,4311,10.5,20.3,1610612751_2021-22,0.537,7,4,2022,201142_2022,0.001,0.298141
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10893,1630637,Carlik Jones,Carlik,1610612743,DEN,24.0,5,3,2,0.600,4.7,0.2,1.8,0.111,0.0,0.2,0.000,0.4,0.4,1.000,0.2,0.4,0.6,1.0,0.6,0.2,0.0,0.0,0.0,0.2,0.8,-4.0,3.0,0,0,2.8,2021-22,1630637_2021-22,94.6,88.7,88.7,122.7,121.8,121.8,-28.1,-33.1,-33.1,0.417,1.67,27.8,0.042,0.069,0.057,16.7,16.8,0.111,0.202,0.220,0.240,107.61,111.44,92.87,111.44,0.016,53,0.2,1.8,1610612743_2021-22,0.585,6,2,2022,1630637_2022,0.000,0.000000
11095,1630206,Jay Scrubb,Jay,1610612746,LAC,21.0,18,8,10,0.444,6.7,1.0,2.6,0.391,0.3,1.2,0.286,0.4,0.6,0.700,0.1,0.8,0.9,0.4,0.4,0.2,0.2,0.0,0.5,0.4,2.7,0.9,5.2,0,0,5.2,2021-22,1630206_2021-22,106.9,105.5,105.5,100.8,98.6,98.6,6.1,6.9,6.9,0.095,1.14,12.3,0.016,0.100,0.060,10.8,10.7,0.457,0.486,0.188,0.194,107.39,109.34,91.12,109.34,0.065,275,1.0,2.6,1610612746_2021-22,0.512,8,3,2022,1630206_2022,0.000,0.000000
11093,1629637,Jaxson Hayes,Jaxson,1610612740,NOP,22.0,70,29,41,0.414,20.0,3.5,5.7,0.616,0.3,0.8,0.351,2.1,2.7,0.766,1.6,2.9,

With rf_model_2.

In [100]:
pred_2022_2 = best_rf_model_2.predict(X_test_2022_tr)

In [101]:
pred_2022_2 = pd.Series(pred_2022_2, index=X_test_2022.index, name='Share_PRED')

In [102]:
output_2022_2 = pd.concat([data_2022, pred_2022_2], axis=1)

In [103]:
output_2022_2.isna().sum().sum()

0

In [104]:
output_2022_2.sort_values(by='Share_PRED', ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,Share_PRED
11028,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,27.0,67,45,22,0.672,32.9,10.3,18.6,0.553,1.1,3.6,0.293,8.3,11.4,0.722,2.0,9.6,11.6,5.8,3.3,1.1,1.4,0.9,3.2,8.8,29.9,5.9,56.5,46,4,53.2,2021-22,203507_2021-22,115.5,115.9,115.9,107.8,107.9,107.9,7.7,8.0,8.0,0.304,1.77,17.8,0.060,0.267,0.168,10.0,10.0,0.582,0.633,0.340,0.347,103.36,103.12,85.93,103.12,0.210,4743,10.3,18.6,1610612749_2021-22,0.622,3,1,2022,203507_2022,0.595,0.552685
11117,203954,Joel Embiid,Joel,1610612755,PHI,28.0,68,45,23,0.662,33.8,9.8,19.6,0.499,1.4,3.7,0.371,9.6,11.8,0.814,2.1,9.6,11.7,4.2,3.1,1.1,1.5,1.2,2.7,8.5,30.6,5.4,55.5,46,2,53.0,2021-22,203954_2021-22,114.6,115.6,115.6,106.9,107.8,107.8,7.7,7.9,7.9,0.236,1.33,13.0,0.068,0.273,0.176,9.8,9.8,0.534,0.616,0.375,0.380,97.81,96.96,80.80,96.96,0.212,4640,9.8,19.6,1610612755_2021-22,0.622,4,2,2022,203954_2022,0.706,0.518553
11271,203999,Nikola Jokic,Nikola,1610612743,DEN,27.0,74,46,28,0.622,33.5,10.3,17.7,0.583,1.3,3.9,0.337,5.1,6.3,0.810,2.8,11.0,13.8,7.9,3.8,1.5,0.9,0.9,2.6,6.5,27.1,6.0,58.6,66,19,54.7,2021-22,203999_2021-22,115.2,117.3,117.3,108.1,108.9,108.9,7.1,8.4,8.4,0.388,2.08,24.8,0.090,0.313,0.209,11.9,11.8,0.620,0.661,0.309,0.316,100.69,99.46,82.88,99.46,0.230,5136,10.3,17.7,1610612743_2021-22,0.585,6,2,2022,203999_2022,0.875,0.501820
11205,1629029,Luka Doncic,Luka,1610612742,DAL,23.0,65,44,21,0.677,35.4,9.9,21.6,0.457,3.1,8.8,0.353,5.6,7.5,0.744,0.9,8.3,9.1,8.7,4.5,1.2,0.6,0.7,2.2,6.2,28.4,2.2,53.1,44,10,52.8,2021-22,1629029_2021-22,112.8,113.8,113.8,108.6,110.3,110.3,4.1,3.5,3.5,0.458,1.95,22.9,0.025,0.230,0.129,11.8,11.8,0.529,0.571,0.368,0.375,97.37,96.18,80.15,96.18,0.191,4604,9.9,21.6,1610612742_2021-22,0.634,4,2,2022,1629029_2022,0.146,0.363387
11196,2544,LeBron James,LeBron,1610612747,LAL,37.0,56,25,31,0.446,37.2,11.4,21.8,0.524,2.9,8.0,0.359,4.5,6.0,0.756,1.1,7.1,8.2,6.2,3.5,1.3,1.1,0.8,2.2,5.5,30.3,-2.1,53.0,21,6,52.3,2021-22,2544_2021-22,110.1,110.9,110.9,112.0,113.5,113.5,-1.9,-2.5,-2.5,0.297,1.78,18.4,0.031,0.185,0.109,10.4,10.2,0.590,0.619,0.317,0.327,102.02,100.92,84.10,100.92,0.180,4380,11.4,21.8,1610612747_2021-22,0.402,11,4,2022,2544_2022,0.001,0.300527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11046,1630171,Isaac Okoro,Isaac,1610612739,CLE,21.0,67,34,33,0.507,29.6,3.1,6.4,0.480,0.8,2.3,0.350,1.8,2.3,0.768,1.1,1.9,3.0,1.8,0.9,0.8,0.3,0.5,2.4,1.9,8.8,1.5,17.7,0,0,16.7,2021-22,1630171_2021-22,112.3,112.4,112.4,108.5,109.3,109.3,3.8,3.1,3.1,0.083,2.09,17.8,0.037,0.065,0.052,8.5,8.6,0.544,0.589,0.123,0.125,97.96,97.54,81.29,97.54,0.057,4014,3.1,6.4,1610612739_2021-22,0.537,8,3,2022,1630171_2022,0.000,0.000016
11048,1630543,Isaiah Jackson,Isaiah,1610612754,IND,20.0,36,9,27,0.250,15.0,3.2,5.7,0.563,0.1,0.4,0.313,1.7,2.4,0.682,1.7,2.5,4.1,0.3,1.1,0.7,1.4,0.3,2.6,2.0,8.3,-1.1,18.9,3,0,17.1,2021-22,1630543_2021-22,112.2,113.3,113.3,116.9,116.1,116.1,-4.7,-2.8,-2.8,0.027,0.25,3.4,0.109,0.157,0.133,13.6,13.6,0.575,0.607,0.214,0.219,103.13,103.00,85.83,103.00,0.093,1158,3.2,5.7,1610612754_2021-22,0.305,13,4,2022,1630543_2022,0.000,0.000016
11049,1630198,Isaiah Joe,Isaiah,1610612755,PHI,22.0,55,35,20,0.636,11.1,1.1,3.2,0.350,0.8,2.5,0.333,0.5,0.6,0.

### Test year 2023.

In [105]:
data_test_2023 = data_test[data_test['YEAR_MVP'] == 2023]
data_test_2023.drop('YEAR_MVP', axis=1, inplace=True)

C:\Users\dcast\AppData\Local\Temp\ipykernel_12032\1874860387.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test_2023.drop('YEAR_MVP', axis=1, inplace=True)


In [106]:
X_test_2023 = data_test_2023.drop('Share', axis=1)
y_test_2023 = data_test_2023['Share']

In [107]:
X_test_2023_tr = pd.DataFrame(scaler.transform(X_test_2023), index=X_test_2023.index, columns=X_test_2023.columns)

In [108]:
pred_2023 = best_rf_model.predict(X_test_2023_tr)

In [109]:
pred_2023 = pd.Series(pred_2023, index=X_test_2023.index, name='Share_PRED')

In [110]:
data_or = pd.read_csv('./data/data.csv')
data_2023 = data_or[data_or['YEAR_MVP'] == 2023]

In [111]:
output_2023 = pd.concat([data_2023, pred_2023], axis=1)

In [112]:
output_2023.isna().sum().sum()

0

In [113]:
output_2023.sort_values(by='Share_PRED', ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,Share_PRED
11593,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,28.0,63,47,16,0.746,32.1,11.2,20.3,0.553,0.7,2.7,0.275,7.9,12.3,0.645,2.2,9.6,11.8,5.7,3.9,0.8,0.8,1.3,3.1,9.0,31.1,5.4,54.8,46,6,52.6,2022-23,203507_2022-23,115.9,116.4,116.4,109.0,109.2,109.2,6.9,7.2,7.2,0.314,1.46,16.2,0.065,0.268,0.171,11.1,11.1,0.572,0.605,0.373,0.383,103.94,103.63,86.36,103.63,0.204,4380,11.2,20.3,1610612749_2022-23,0.707,1,1,2023,203507_2023,0.606,0.669105
11679,203954,Joel Embiid,Joel,1610612755,PHI,29.0,66,43,23,0.652,34.6,11.0,20.1,0.548,1.0,3.0,0.330,10.0,11.7,0.857,1.7,8.4,10.2,4.2,3.4,1.0,1.7,1.4,3.1,7.7,33.1,6.4,56.2,39,1,53.8,2022-23,203954_2022-23,118.3,119.0,119.0,108.6,110.2,110.2,9.7,8.8,8.8,0.233,1.21,12.7,0.057,0.243,0.157,10.5,10.4,0.573,0.655,0.370,0.377,98.41,97.34,81.12,97.34,0.213,4639,11.0,20.1,1610612755_2022-23,0.659,3,2,2023,203954_2023,0.915,0.613762
11826,203999,Nikola Jokic,Nikola,1610612743,DEN,28.0,69,48,21,0.696,33.7,9.4,14.8,0.632,0.8,2.2,0.383,4.9,6.0,0.822,2.4,9.4,11.8,9.8,3.6,1.3,0.7,0.7,2.5,5.7,24.5,9.3,55.7,58,29,50.9,2022-23,203999_2022-23,122.7,124.2,124.2,110.8,111.6,111.6,11.9,12.5,12.5,0.412,2.74,32.1,0.080,0.283,0.186,11.7,11.6,0.660,0.701,0.263,0.268,101.40,100.42,83.68,100.42,0.211,4873,9.4,14.8,1610612743_2022-23,0.646,1,1,2023,203999_2023,0.674,0.449121
11665,1628369,Jayson Tatum,Jayson,1610612738,BOS,25.0,74,52,22,0.703,36.9,9.8,21.1,0.466,3.2,9.3,0.350,7.2,8.4,0.854,1.1,7.7,8.8,4.6,2.9,1.1,0.7,0.8,2.2,5.4,30.1,6.4,49.9,31,1,50.2,2022-23,1628369_2022-23,118.2,119.2,119.2,109.5,110.8,110.8,8.7,8.5,8.5,0.205,1.61,14.5,0.029,0.200,0.117,9.0,8.9,0.543,0.607,0.319,0.327,100.77,99.76,83.13,99.76,0.167,5674,9.8,21.1,1610612738_2022-23,0.695,2,1,2023,1628369_2023,0.280,0.437743
11768,1629029,Luka Doncic,Luka,1610612742,DAL,24.0,66,33,33,0.500,36.2,10.9,22.0,0.496,2.8,8.2,0.342,7.8,10.5,0.742,0.8,7.8,8.6,8.0,3.6,1.4,0.5,0.7,2.5,7.3,32.4,1.9,56.8,36,10,55.6,2022-23,1629029_2022-23,116.7,118.1,118.1,114.5,116.0,116.0,2.2,2.1,2.1,0.408,2.24,21.1,0.024,0.224,0.124,9.4,9.4,0.560,0.609,0.368,0.376,98.80,97.63,81.36,97.63,0.202,4874,10.9,22.0,1610612742_2022-23,0.463,11,3,2023,1629029_2023,0.010,0.430286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11638,1630552,Jalen Johnson,Jalen,1610612737,ATL,21.0,70,35,35,0.500,14.9,2.3,4.6,0.491,0.4,1.5,0.288,0.7,1.1,0.628,0.7,3.3,4.0,1.2,0.6,0.5,0.5,0.3,1.6,0.8,5.6,-0.1,14.8,1,0,13.3,2022-23,1630552_2022-23,111.5,111.1,111.1,111.0,110.8,110.8,0.5,0.3,0.3,0.106,2.02,17.3,0.047,0.214,0.131,8.6,8.5,0.537,0.554,0.156,0.161,103.06,103.32,86.10,103.32,0.099,2234,2.3,4.6,1610612737_2022-23,0.500,8,2,2023,1630552_2023,0.000,0.000000
11633,1631222,Jake LaRavia,Jake,1610612763,MEM,21.0,35,22,13,0.629,11.8,1.1,2.7,0.389,0.7,2.0,0.338,0.2,0.3,0.778,0.5,1.3,1.8,0.6,0.5,0.3,0.1,0.1,1.4,0.3,3.0,-2.2,6.9,0,0,7.0,2022-23,1631222_2022-23,107.4,106.4,106.4,114.1,113.9,113.9,-6.7,-7.6,-7.6,0.068,1.17,15.2,0.044,0.100,0.072,13.0,13.0,0.516,0.531,0.117,0.120,101.86,102.42,85.35,102.42,0.040,879,1.1,2.7,1610612763_2022-23,0.622,2,1,2023,1631222_2023,0.000,0.000000
11632,1630256,Jae'Sean Tate,Jae'Sean,1610612745,HOU,27.0,31,7,24,0.226,21.9,3.5,7.4,0.480,0.4,1.5,0.283

With rf_model_2.

In [114]:
pred_2023_2 = best_rf_model_2.predict(X_test_2023_tr)

In [115]:
pred_2023_2 = pd.Series(pred_2023_2, index=X_test_2023.index, name='Share_PRED')

In [116]:
output_2023_2 = pd.concat([data_2023, pred_2023_2], axis=1)

In [117]:
output_2023_2.isna().sum().sum()

0

In [118]:
output_2023_2.sort_values(by='Share_PRED', ascending=False)

,PLAYER_ID,PLAYER_NAME,NICKNAME,TEAM_ID,TEAM_ABBREVIATION,AGE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,NBA_FANTASY_PTS,DD2,TD3,WNBA_FANTASY_PTS,SEASON,PK,E_OFF_RATING,OFF_RATING,sp_work_OFF_RATING,E_DEF_RATING,DEF_RATING,sp_work_DEF_RATING,E_NET_RATING,NET_RATING,sp_work_NET_RATING,AST_PCT,AST_TO,AST_RATIO,OREB_PCT,DREB_PCT,REB_PCT,TM_TOV_PCT,E_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,sp_work_PACE,PIE,POSS,FGM_PG,FGA_PG,PK_2,WIN_PCT,CONF_RANK,DIV_RANK,YEAR_MVP,PK_3,Share,Share_PRED
11593,203507,Giannis Antetokounmpo,Giannis,1610612749,MIL,28.0,63,47,16,0.746,32.1,11.2,20.3,0.553,0.7,2.7,0.275,7.9,12.3,0.645,2.2,9.6,11.8,5.7,3.9,0.8,0.8,1.3,3.1,9.0,31.1,5.4,54.8,46,6,52.6,2022-23,203507_2022-23,115.9,116.4,116.4,109.0,109.2,109.2,6.9,7.2,7.2,0.314,1.46,16.2,0.065,0.268,0.171,11.1,11.1,0.572,0.605,0.373,0.383,103.94,103.63,86.36,103.63,0.204,4380,11.2,20.3,1610612749_2022-23,0.707,1,1,2023,203507_2023,0.606,0.756927
11679,203954,Joel Embiid,Joel,1610612755,PHI,29.0,66,43,23,0.652,34.6,11.0,20.1,0.548,1.0,3.0,0.330,10.0,11.7,0.857,1.7,8.4,10.2,4.2,3.4,1.0,1.7,1.4,3.1,7.7,33.1,6.4,56.2,39,1,53.8,2022-23,203954_2022-23,118.3,119.0,119.0,108.6,110.2,110.2,9.7,8.8,8.8,0.233,1.21,12.7,0.057,0.243,0.157,10.5,10.4,0.573,0.655,0.370,0.377,98.41,97.34,81.12,97.34,0.213,4639,11.0,20.1,1610612755_2022-23,0.659,3,2,2023,203954_2023,0.915,0.673698
11826,203999,Nikola Jokic,Nikola,1610612743,DEN,28.0,69,48,21,0.696,33.7,9.4,14.8,0.632,0.8,2.2,0.383,4.9,6.0,0.822,2.4,9.4,11.8,9.8,3.6,1.3,0.7,0.7,2.5,5.7,24.5,9.3,55.7,58,29,50.9,2022-23,203999_2022-23,122.7,124.2,124.2,110.8,111.6,111.6,11.9,12.5,12.5,0.412,2.74,32.1,0.080,0.283,0.186,11.7,11.6,0.660,0.701,0.263,0.268,101.40,100.42,83.68,100.42,0.211,4873,9.4,14.8,1610612743_2022-23,0.646,1,1,2023,203999_2023,0.674,0.635847
11665,1628369,Jayson Tatum,Jayson,1610612738,BOS,25.0,74,52,22,0.703,36.9,9.8,21.1,0.466,3.2,9.3,0.350,7.2,8.4,0.854,1.1,7.7,8.8,4.6,2.9,1.1,0.7,0.8,2.2,5.4,30.1,6.4,49.9,31,1,50.2,2022-23,1628369_2022-23,118.2,119.2,119.2,109.5,110.8,110.8,8.7,8.5,8.5,0.205,1.61,14.5,0.029,0.200,0.117,9.0,8.9,0.543,0.607,0.319,0.327,100.77,99.76,83.13,99.76,0.167,5674,9.8,21.1,1610612738_2022-23,0.695,2,1,2023,1628369_2023,0.280,0.511871
11768,1629029,Luka Doncic,Luka,1610612742,DAL,24.0,66,33,33,0.500,36.2,10.9,22.0,0.496,2.8,8.2,0.342,7.8,10.5,0.742,0.8,7.8,8.6,8.0,3.6,1.4,0.5,0.7,2.5,7.3,32.4,1.9,56.8,36,10,55.6,2022-23,1629029_2022-23,116.7,118.1,118.1,114.5,116.0,116.0,2.2,2.1,2.1,0.408,2.24,21.1,0.024,0.224,0.124,9.4,9.4,0.560,0.609,0.368,0.376,98.80,97.63,81.36,97.63,0.202,4874,10.9,22.0,1610612742_2022-23,0.463,11,3,2023,1629029_2023,0.010,0.448736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11621,203210,JaMychal Green,JaMychal,1610612744,GSW,33.0,57,30,27,0.526,14.0,2.4,4.4,0.540,0.7,1.9,0.378,0.9,1.2,0.776,1.3,2.3,3.6,0.9,0.9,0.4,0.4,0.2,1.8,1.0,6.4,-2.7,13.7,0,0,13.3,2022-23,203210_2022-23,108.7,108.3,108.3,115.9,116.4,116.4,-7.2,-8.2,-8.2,0.094,0.98,13.3,0.087,0.164,0.125,13.5,13.5,0.623,0.650,0.169,0.174,105.28,105.25,87.71,105.25,0.097,1741,2.4,4.4,1610612744_2022-23,0.537,6,4,2023,203210_2023,0.000,0.000016
11619,1630550,JT Thor,JT,1610612766,CHA,20.0,69,22,47,0.319,14.0,1.4,3.4,0.399,0.6,1.8,0.317,0.5,0.7,0.702,0.5,1.7,2.2,0.5,0.6,0.3,0.3,0.2,1.0,0.5,3.8,-2.4,8.5,0,0,8.4,2022-23,1630550_2022-23,103.6,102.7,102.7,110.4,110.2,110.2,-6.8,-7.5,-7.5,0.054,0.82,10.6,0.035,0.115,0.075,13.0,13.0,0.481,0.506,0.130,0.134,101.91,102.44,85.37,102.44,0.051,2064,1.4,3.4,1610612766_2022-23,0.329,14,5,2023,1630550_2023,0.000,0.000016
11618,1631120,JD Davison,JD,1610612738,BOS,20.0,12,11,1,0.917,5.5,0.7,1.6,0.421,0.2,0.6,0.286,0.1,0.2,